In [15]:
#Import dependencies
import pandas as pd
import numpy as np
import requests
import time
import csv
import json
from pprint import pprint

# Import API key
from api_keys import api_key

#bring in all the necessary csv file data generated based on Francis Wong's code and Eric Lieu's code
df_2015 = pd.read_csv("2015_Top30zipcodes.csv")
df_2015 = df_2015[['Zip Code','Mean', 'Median']]

df_2016 = pd.read_csv("2016_Top30zipcodes.csv")
df_2016 = df_2016[['Zip Code','Mean', 'Median']]

df_2017 = pd.read_csv("2017_Top30codes")
df_2017 = df_2017[['Zip Code','Mean', 'Median']]

df_2018 = pd.read_csv("2018_Top30zipcodes_edited.csv")
df_2018 = df_2018[['Zip Code','Mean', 'Median']]

#base file is a data frame containing the preliminary data
base_file = pd.read_csv("consolidated3.csv")
base_file = base_file.set_index('Zip Codes')
#drop the 'Count' column
base_file = base_file.drop(['Count'],axis=1)
#get a list of the precincts
base_file

,Borough,Neighborhood,Precinct
Zip Codes,,,
11206,Brooklyn,Bushwick and Williamsburg,90
11221,Brooklyn,Bushwick and Williamsburg,83
11237,Brooklyn,Bushwick and Williamsburg,90
11216,Brooklyn,Central Brooklyn,79
11233,Brooklyn,Central Brooklyn,81
11238,Brooklyn,Central Brooklyn,77
10027,Manhattan,Central Harlem,28
10011,Manhattan,Chelsea and Clinton,10
10019,Manhattan,Chelsea and Clinton,18


In [16]:
#Based on Amanda Wishnie's code; merge 2015 and 2016 zip code and neighborhood data by zipcode
df_1516 = pd.merge(df_2015, df_2016, on="Zip Code", how="outer", suffixes=("_2015", "_2016"))
df_151617 = pd.merge(df_1516, df_2017, on="Zip Code", how="outer")
df_151617 = df_151617.rename(columns={'Mean':'Mean_2017','Median':'Median_2017'})

df_all = pd.merge(df_151617, df_2018, on="Zip Code", how="outer")
df_all = df_all.rename(columns={'Mean':'Mean_2018','Median':'Median_2018'})
df_all = df_all.rename(columns={'Zip Code':'Zip Codes'})
df_all = df_all.set_index('Zip Codes')
df_all = df_all.dropna()
df_all

,Mean_2015,Median_2015,Mean_2016,Median_2016,Mean_2017,Median_2017,Mean_2018,Median_2018
Zip Codes,,,,,,,,
11211,150.991682,129.0,143.768193,119.0,145.759296,113.0,146.840053,110.0
10002,182.400477,150.0,168.815366,135.0,174.141667,129.0,171.128160,130.0
10009,177.778549,150.0,165.071030,135.0,166.544186,135.0,170.691880,140.0
10003,220.328891,180.0,218.895993,169.0,220.491229,160.0,223.038827,169.0
11206,102.391970,80.0,94.994238,75.0,94.821426,75.0,96.825835,75.0
11238,144.082211,120.0,138.259867,106.0,135.210127,100.0,137.844509,105.0
11221,95.713076,75.0,89.587325,70.0,88.334267,69.0,87.629931,68.0
10011,242.854346,200.0,225.796276,199.0,225.691729,195.0,235.382632,190.0
11216,111.900207,90.0,110.022522,85.0,107.358035,85.0,110.636435,85.0


In [17]:
#Based on Amanda Wishnie's code; consolidate 2016 to 2018 code
df_consol = pd.merge(df_all, base_file, on="Zip Codes", how="outer")
#format float format precinct data to no decimal integers
df_consol['Precinct'] = df_consol['Precinct'].map('{:,.0f}'.format)
#groupby three indexes
neighborhood_gb = df_consol.groupby(['Neighborhood','Zip Codes','Precinct']).mean()
neighborhood_df = pd.DataFrame(neighborhood_gb)
#drop all rows with NaN values
neighborhood_df = neighborhood_df.dropna()
neighborhood_df

Mean_2015  Median_2015  \
Neighborhood                  Zip Codes Precinct                            
Bushwick and Williamsburg     11206     90        102.391970         80.0   
                              11221     83         95.713076         75.0   
                              11237     90         91.916442         75.0   
Central Brooklyn              11216     79        111.900207         90.0   
                              11238     77        144.082211        120.0   
Central Harlem                10027     28        128.709048        110.0   
Chelsea and Clinton           10011     10        242.854346        200.0   
                              10019     18        229.525049        189.0   
                              10036     18        218.437125        175.0   
East Harlem                   10029     23        126.461440        100.0   
Flatbush                      11226     70         92.680582         75.0   
Gramercy Park and Murray Hill 10016     17        237.572914        185.0   
Greenpoint                    11211     94        150.991682        129.0   
                              11222     94        140.432536        120.0   
Greenwich Village and Soho    10012     1         255.219760        200.0   
                              10014     6         252.500823        210.0   
Inwood and Washington Heights 10031     30        104.395633         80.0   
Lower East Side               10002     7         182.400477        150.0   
                              10003     9         220.328891        180.0   
                              10009     9         177.778549        150.0   
Northwest Brooklyn            11205     79        165.375414        115.0   
                              11215     78        162.932613        139.0   
                              11217     84        165.974973        140.0   
Upper West Side               10023     20        234.262821        190.0   
                              10025     24        183.937929        145.0   

                                                   Mean_2016  Median_2016  \
Neighborhood                  Zip Codes Precinct                            
Bushwick and Williamsburg     11206     90         94.994238         75.0   
                              11221     83         89.587325         70.0   
                              11237     90         86.114981         69.0   
Central Brooklyn              11216     79        110.022522         85.0   
                              11238     77        138.259867        106.0   
Central Harlem                10027     28        121.078350        100.0   
Chelsea and Clinton           10011     10        225.796276        199.0   
                              10019     18        215.508512        175.0   
                              10036     18        212.077208        165.0   
East Harlem                   10029     23        127.426550        100.0   
Flatbush                      11226     70         84.754190         70.0   
Gramercy Park and Murray Hill 10016     17        214.245595        175.0   
Greenpoint                    11211     94        143.768193        119.0   
                              11222     94        128.623164        109.0   
Greenwich Village and Soho    10012     1         230.587597        190.0   
                              10014     6         239.303179        200.0   
Inwood and Washington Heights 10031     30         99.883374         75.0   
Lower East Side               10002     7         168.815366        135.0   
                              10003     9         218.895993        169.0   
                              10009     9         165.071030        135.0   
Northwest Brooklyn            11205     79        154.145494        100.0   
                              11215     78        150.024911        130.0   
                              11217     84        164.352042        130.0   
Upper West Side               10023     20    

In [4]:
#groupby precinct first
neighborhood_gb = df_consol.groupby(['Precinct','Neighborhood','Zip Codes',]).mean()
neighborhood_df = pd.DataFrame(neighborhood_gb)
#drop NaN values
neighborhood_df = neighborhood_df.dropna()

#sort by ascending alphabetical 'Neighborhood' column
neighborhood_df.sort_index(axis=0, level='Neighborhood', ascending=True)

Mean_2015  Median_2015  \
Precinct Neighborhood                  Zip Codes                            
83       Bushwick and Williamsburg     11221       95.713076         75.0   
90       Bushwick and Williamsburg     11206      102.391970         80.0   
                                       11237       91.916442         75.0   
77       Central Brooklyn              11238      144.082211        120.0   
79       Central Brooklyn              11216      111.900207         90.0   
28       Central Harlem                10027      128.709048        110.0   
10       Chelsea and Clinton           10011      242.854346        200.0   
18       Chelsea and Clinton           10019      229.525049        189.0   
                                       10036      218.437125        175.0   
23       East Harlem                   10029      126.461440        100.0   
70       Flatbush                      11226       92.680582         75.0   
17       Gramercy Park and Murray Hill 10016      237.572914        185.0   
94       Greenpoint                    11211      150.991682        129.0   
                                       11222      140.432536        120.0   
1        Greenwich Village and Soho    10012      255.219760        200.0   
6        Greenwich Village and Soho    10014      252.500823        210.0   
30       Inwood and Washington Heights 10031      104.395633         80.0   
7        Lower East Side               10002      182.400477        150.0   
9        Lower East Side               10003      220.328891        180.0   
                                       10009      177.778549        150.0   
78       Northwest Brooklyn            11215      162.932613        139.0   
79       Northwest Brooklyn            11205      165.375414        115.0   
84       Northwest Brooklyn            11217      165.974973        140.0   
20       Upper West Side               10023      234.262821        190.0   
24       Upper West Side               10025      183.937929        145.0   

                                                   Mean_2016  Median_2016  \
Precinct Neighborhood                  Zip Codes                            
83       Bushwick and Williamsburg     11221       89.587325         70.0   
90       Bushwick and Williamsburg     11206       94.994238         75.0   
                                       11237       86.114981         69.0   
77       Central Brooklyn              11238      138.259867        106.0   
79       Central Brooklyn              11216      110.022522         85.0   
28       Central Harlem                10027      121.078350        100.0   
10       Chelsea and Clinton           10011      225.796276        199.0   
18       Chelsea and Clinton           10019      215.508512        175.0   
                                       10036      212.077208        165.0   
23       East Harlem                   10029      127.426550        100.0   
70       Flatbush                      11226       84.754190         70.0   
17       Gramercy Park and Murray Hill 10016      214.245595        175.0   
94       Greenpoint                    11211      143.768193        119.0   
                                       11222      128.623164        109.0   
1        Greenwich Village and Soho    10012      230.587597        190.0   
6        Greenwich Village and Soho    10014      239.303179        200.0   
30       Inwood and Washington Heights 10031       99.883374         75.0   
7        Lower East Side               10002      168.815366        135.0   
9        Lower East Side               10003      218.895993        169.0   
                                       10009      165.071030        135.0   
78       Northwest Brooklyn            11215      150.024911        130.0   
79       Northwest Brooklyn            11205      154.145494        100.0   
84       Northwest Brooklyn            11217      164.352042        130.0   
20       Upper West Side               10023  

In [18]:
#API query url for data on NYPD complaints reports of violations, misdemeanors, and felonies
url = "https://data.cityofnewyork.us/resource/9s4h-37hy.json?"
#precint query 
precinct = "addr_pct_cd="
#report date query
report_date = "rpt_dt="

#counts the number of police reports by year and precinct
def crime_search(year,x):
    #makes 'counter' into a global variable that can be used outside of the function
    global counter
    counter = 0 
    #loops through every month of the year
    for month in range(1,13):
        for day in range(1,32):
            query_url = (f"{url}$$app_token={api_key}&{precinct}{x}&{report_date}{year}-{month}-{day}")
            response = requests.get(query_url)
            text = str(response)
            json = requests.get(query_url).json()
            if text == "<Response [200]>":
                for entry in json:
                    counter += 1
                    # end ="\r" makes it so that below processing log prints over the same line
                    print(f"Processing Report {counter}", end="\r"),
            elif text == "<Response [400]>": 
                counter += 0
    return counter


In [4]:
#selected precincts from the list of 30 for which data is retrieved
precincts = [83, 77, 28, 10, 23, 70, 17, 94, 30, 79]
precincts

[83, 77, 28, 10, 23, 70, 17, 94, 30, 79]

In [7]:
report_count_2015 = []
#for loop running the crime_search function to count the number of police reports filed at given precinct for that year
for prc in precincts:
    crime_search(2015,prc)
    print(f"There were {counter} police reports filed at Precinct {prc} in 2015.")
    report_count_2015.append(counter)
    counter == 0

There were 6236 police reports filed at Precinct 83 in 2015.
There were 6513 police reports filed at Precinct 77 in 2015.
There were 4956 police reports filed at Precinct 28 in 2015.
There were 3786 police reports filed at Precinct 10 in 2015.
There were 6403 police reports filed at Precinct 23 in 2015.
There were 7615 police reports filed at Precinct 70 in 2015.
There were 2961 police reports filed at Precinct 17 in 2015.
There were 3658 police reports filed at Precinct 94 in 2015.
There were 4204 police reports filed at Precinct 30 in 2015.
There were 6768 police reports filed at Precinct 79 in 2015.


In [8]:
#start building the csv file with the headers
csv_columns = ['Precinct','Report Count','Year']
with open('2015_report_count_precinct.csv', mode='w') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(csv_columns)
#append corresponding precinct and police report account information in the same line into the csv file
x=0
for count in report_count_2015:
    prec_num = precincts[x]
    report_count = report_count_2015[x]
    year = "2015"
    data_list = [prec_num, report_count, year]
    with open('2015_report_count_precinct.csv',mode='a') as csv_file:
        csv_file = csv.writer(csv_file, delimiter=',')
        csv_file.writerow(data_list)
    x += 1

In [9]:
report_count_2016 = []
#see above
for prc in precincts:
    crime_search(2016,prc)
    print(f"There were {counter} police reports filed at Precinct {prc} in 2016.")
    report_count_2016.append(counter)
    counter == 0
    

There were 6442 police reports filed at Precinct 83 in 2016.
There were 6310 police reports filed at Precinct 77 in 2016.
There were 5674 police reports filed at Precinct 28 in 2016.
There were 3907 police reports filed at Precinct 10 in 2016.
There were 6469 police reports filed at Precinct 23 in 2016.
There were 7356 police reports filed at Precinct 70 in 2016.
There were 3290 police reports filed at Precinct 17 in 2016.
There were 3719 police reports filed at Precinct 94 in 2016.
There were 4381 police reports filed at Precinct 30 in 2016.
There were 6950 police reports filed at Precinct 79 in 2016.


In [10]:
csv_columns = ['Precinct','Report Count','Year']
with open('2016_report_count_precinct.csv', mode='w') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(csv_columns)
x=0
for count in report_count_2016:
    prec_num = precincts[x]
    report_count = report_count_2016[x]
    year = "2016"
    data_list = [prec_num, report_count, year]
    with open('2016_report_count_precinct.csv',mode='a') as csv_file:
        csv_file = csv.writer(csv_file, delimiter=',')
        csv_file.writerow(data_list)
    x += 1

In [11]:
#see above
report_count_2017 = []

for prc in precincts:
    crime_search(2017,prc)
    print(f"There were {counter} police reports filed at Precinct {prc} in 2017.")
    report_count_2017.append(counter)
    counter == 0

There were 6538 police reports filed at Precinct 83 in 2017.
There were 6157 police reports filed at Precinct 77 in 2017.
There were 5710 police reports filed at Precinct 28 in 2017.
There were 3670 police reports filed at Precinct 10 in 2017.
There were 6207 police reports filed at Precinct 23 in 2017.
There were 7198 police reports filed at Precinct 70 in 2017.
There were 3729 police reports filed at Precinct 17 in 2017.
There were 3740 police reports filed at Precinct 94 in 2017.
There were 3580 police reports filed at Precinct 30 in 2017.
There were 7021 police reports filed at Precinct 79 in 2017.


In [12]:
#see above
csv_columns = ['Precinct','Report Count','Year']
with open('2017_report_count_precinct.csv', mode='w') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(csv_columns)
x=0
for count in report_count_2017:
    prec_num = precincts[x]
    report_count = report_count_2017[x]
    year = "2017"
    data_list = [prec_num, report_count, year]
    with open('2017_report_count_precinct.csv',mode='a') as csv_file:
        csv_file = csv.writer(csv_file, delimiter=',')
        csv_file.writerow(data_list)
    x += 1

There were 3154 police reports filed at Precinct 83 in 2018.
There were 3079 police reports filed at Precinct 77 in 2018.
There were 2722 police reports filed at Precinct 28 in 2018.
There were 1914 police reports filed at Precinct 10 in 2018.
